## File SDK Examples

This notebook demonstrates how to work with Evo Files using the **Evo Python SDK**, providing a simplified and streamlined experience for common file operations.

### SDK vs API Approach

This notebook uses the **File SDK (FileAPIClient)** which:
- ✅ Simplified method calls with minimal boilerplate code
- ✅ Automatic handling of complex API interactions
- ✅ Better error handling and validation
- ✅ Type hints and IDE autocomplete support
- ✅ Recommended for most common use cases

### Want More Control Over API Requests?

If you need granular control over API requests and want to see detailed response structures, check out the `api-examples.ipynb` notebook in this same directory. The API examples show:
- Direct API calls using the connector
- Full control over request/response handling
- Detailed API response structures and metadata
- Useful for debugging and advanced use cases

In [ ]:
from evo.files import FileAPIClient
from evo.notebooks import ServiceManagerWidget

input_location = "sample-data"

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

manager = await ServiceManagerWidget.with_auth_code(
    redirect_url=redirect_url,
    client_id=client_id,
).login()

### Create the File API client

In [ ]:
# Get the environment and connector from the ServiceManagerWidget instance.
# The environment contains the hub URL, organization ID, and workspace ID.
# The connector is used to make API calls to the Evo service.
environment = manager.get_environment()
connector = manager.get_connector()

file_client = FileAPIClient(
    connector=connector,
    environment=environment,
)

## File operations

### List all files in the workspace.

In [ ]:
import pandas as pd

from evo.common import Page
from evo.files import FileMetadata

try:
    paginated_files: Page[FileMetadata] = await file_client.list_all_files()

    # Create file list from paginated results
    file_list = [
        {"Name": file.name, "Path": file.path, "ID": file.id, "Version_id": file.version_id} for file in paginated_files
    ]

    df = pd.DataFrame(file_list)
    display(df)
except Exception as e:
    print(f"Error listing files:\n{e}")

### List paginated files

If you have access to more than 50 files, it may be more convenient to work with a paginated list of files. By changing the `limit` and `offset` values each time to make the request, you can efficiently iterate through your files.

In [ ]:
import pandas as pd

from evo.common import Page
from evo.files import FileMetadata

try:
    paginated_files: Page[FileMetadata] = await file_client.list_files(limit=5, offset=0)

    # Create file list from paginated results
    file_list = [
        {"Name": file.name, "Path": file.path, "ID": file.id, "Version_id": file.version_id}
        for file in paginated_files.items()
    ]

    df = pd.DataFrame(file_list)
    display(df)
except Exception as e:
    print(f"Error listing files:\n{e}")

### Upload/update a file by path

In [ ]:
path = "example.csv"

# Prepare an upload context by path.
ctx = await file_client.prepare_upload_by_path(path)

# Use the upload context to upload the file.
await ctx.upload_from_path("sample-data/example.txt", connector.transport)

### Fetch details for a specific file

In [ ]:
import pandas as pd

from evo.files import FileMetadata

try:
    file: FileMetadata = await file_client.get_file_by_path(path=path)

    # Display the file details
    file_info = {"Name": file.name, "Path": file.path, "ID": file.id, "Version_id": file.version_id}

    df = pd.DataFrame([file_info]).T
    df.columns = ["File info"]
    display(df)
except Exception as e:
    print(f"Error listing files:\n{e}")

### Download a file

If you need to download a specific version of a file, you can include `version={version_id}` in your request parameters.

In [ ]:
ctx = await file_client.prepare_download_by_path(path)

# Use the download context to download the file.
file_location = await ctx.download_to_cache(manager.cache, connector.transport)
print(f"File downloaded to {str(file_location)}")

### Delete a file

When you delete a file, it will be soft-deleted. Deleted files will no longer be accessible through `list_all_files` by default.
The `delete_file_by_id` function has no return value.

In [ ]:
try:
    await file_client.delete_file_by_id(file_id=file_info["ID"])
    print(f"Successfully deleted file with ID: {file_info['ID']}")
except Exception as e:
    print(f"Error deleting file with ID {file_info['ID']}: {e}")

### Fetch details of a deleted file

You must include the `deleted=True` parameter to find details of a deleted file.

In [ ]:
import pandas as pd

from evo.files import FileMetadata

try:
    file: FileMetadata = await file_client.get_file_by_id(file_id=file_info["ID"], deleted=True)

    # Display the file details
    file_info = {"Name": file.name, "Path": file.path, "ID": file.id, "Version_id": file.version_id}

    df = pd.DataFrame([file_info]).T
    df.columns = ["File info"]
    display(df)
except Exception as e:
    print(f"Error listing files:\n{e}")

### Restore a deleted file

The `restore_file_by_id` method can return two different responses:
- **`None`** (HTTP 204): File restored without location change
- **`FileMetadata`** (HTTP 303): File restored with location change (e.g., renamed during restore)


In [ ]:
try:
    result = await file_client.restore_file_by_id(file_id=file_info["ID"])

    if result is None:
        # HTTP 204: File restored without location change
        print(f"Successfully restored file with ID: {file_info['ID']}")
    else:
        # HTTP 303: File restored with location change
        print(f"Successfully restored file with ID: {file_info['ID']}")
        print("File path changed.")
        print(f"New File Path: {result.path}")
        print(f"New File Name: {result.name}")
except Exception as e:
    print(f"Error restoring file with ID {file_info['ID']}: {e}")